In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)

from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner

In [2]:
gs.destroy()
gs.init(backend=gs.cpu)
gs.set_random_seed(seed=42)
#config = {'viewr_option' : {'width': 640, 'height': 480}, 'box_box_detection': True}
adapter = GenesisAdapter()
scene = adapter.scene

[Genesis] [12:41:49] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [12:41:49] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [12:41:49] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [12:41:50] [INFO] Running on [13th Gen Intel(R) Core(TM) i7-13800H] with backend gs.cpu. Device memory: 31.59 GB.
[Genesis] [12:41:50] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [12:41:51] [INFO] Scene <e1a7944> created.


In [3]:
plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='../model/panda_vir.xml'),
)
                          
# table = scene.add_entity(
#     gs.morphs.MJCF(file='../model/common_arena/tables.xml'),
# )

table1 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0 ,
        friction=5.0
    )
)
        
table2 = scene.add_entity(
    gs.morphs.Box(
        size = (0.7, 1.7, 0.02),
        pos = (-0.5, 0, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table3 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, -0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

table4 = scene.add_entity(
    gs.morphs.Box(
        size = (0.3, 0.7, 0.02),
        pos = (0, 0.5, 0.18),
        fixed = True,
    ),
    surface=gs.surfaces.Default(
        color = (0.5, 0.5, 0.5),
    ),
    material=gs.materials.Rigid(
        gravity_compensation=1.0,
        friction=5.0
    )
)

bottle = scene.add_entity(
    material=gs.materials.Rigid(rho=300),
    morph=gs.morphs.URDF(
        file="urdf/3763/mobility_vhacd.urdf",
        scale=0.09,
        pos=(0.65, 0.0, 0.036 + 0.19),
        quat=(-np.pi/2, 0, np.pi/2, 0),
    ),
    # visualize_contact=True,
)

# bottle = scene.add_entity(
#     morph = gs.morphs.URDF(
#         file="../assets/urdf/bottle/bottle.urdf",
#         scale=0.1,
#         pos=(0.65, 0.4, 0.036 + 0.19),
#         quat=(-np.pi/2, 0, np.pi/2, 0),
#     ),
# )
cube_size = (0.07, 0.1, 0.1)
cube_pos = (0.4, 0.4, 0.05+0.19)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
    ),
    surface=gs.surfaces.Default(
        color = (0.8, 0.1 , 0.1),   
    )
)
cube.surface.color = (0.8, 0.1 , 0.1)

cylinder_pos = (0.7, -0.2, 0.03+0.19)
cylinder_quat = (0, -np.sqrt(2)/2, 0, np.sqrt(2)/2)
cylinder_radius = 0.03
cylinder_height = 0.15

cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius = cylinder_radius,
        height = cylinder_height,
        pos    = cylinder_pos,
        quat   = cylinder_quat,
    ),
    surface=gs.surfaces.Default(
        color = (0.1, 0.8 , 0.1),   
    )
)
cylinder.surface.color = (0.2, 0.8, 0.2)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)

[Genesis] [12:41:56] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <1ad26f1>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [12:41:56] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <b015e1e>, morph: <gs.morphs.MJCF(file='d:\GitHubRepository\pandaSim\model\panda_vir.xml')>, material: <gs.materials.Rigid>.
[Genesis] [12:41:58] [WARNING] (MJCF) Tendon not supported
[Genesis] [12:41:58] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [12:41:58] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [12:41:58] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [12:41:58] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [12:41:59] [INFO] Adding <gs.RigidEntity>. idx: 2, uid: <9927a57>, morph:

In [4]:
n_envs = 9
scene.build(n_envs=n_envs, env_spacing=(2, 2), center_envs_at_origin=True )

[Genesis] [12:41:59] [INFO] Building scene <e1a7944>...
[Genesis] [12:42:01] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [12:42:12] [INFO] Compiling simulation kernels...
[Genesis] [12:49:06] [INFO] Building visualizer...
[Genesis] [12:49:11] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
import random

def random_dual_range(range1,range2, size):
    a, b = range1
    c, d = range2
    if random.random() < 0.5:
        return np.random.uniform(a, b, size=size)
    else:
        return np.random.uniform(c, d, size=size)
    
def multiply_quaternion(q1, q2):
    w1, x1, y1, z1 = q1[:, 0], q1[:, 1], q1[:, 2], q1[:, 3]
    w2, x2, y2, z2 = q2[:, 0], q2[:, 1], q2[:, 2], q2[:, 3]
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2

    result = np.stack([w, x, y, z], axis=1)  # (N, 4)
    return result   

In [6]:
scene.reset()

[Genesis] [13:25:34] [INFO] Resetting Scene <e1a7944>.


In [7]:
# Cube randomization
cube_pq = adapter.get_pose(cube)
cube_pq[:, 0] = np.random.uniform(0.2, 0.5, size=(n_envs,))
cube_pq[:, 1] = np.random.uniform(0.3, 0.55, size=(n_envs,))
cube_theta = np.random.uniform(-np.pi/6, np.pi/6, size=(n_envs,))
cube_pq[:,3] = np.cos(cube_theta)
cube_pq[:,4] = cube_pq[:,5] = np.zeros(n_envs)
cube_pq[:, 6] =  np.sin(cube_theta)


In [8]:
# cylinder randomization
cylinder_pq = adapter.get_pose(cylinder)
cylinder_pq[:, 0] = random_dual_range([0.2, 0.45], [-0.2, -0.45], size=(n_envs,))
cylinder_pq[:, 1] = np.random.uniform(-0.3, -0.55, size=(n_envs,))
cyl_theta = np.random.uniform(-np.pi/6, np.pi/6, size=(n_envs,))
cylinder_pq[:,3] = np.cos(cyl_theta)
cylinder_pq[:,4] =  np.sin(cyl_theta)
cylinder_pq[:,5] = cylinder_pq[:,6] = np.zeros(n_envs)
cylinder_pq[:,3:] = multiply_quaternion(np.array(cylinder.get_quat()), cylinder_pq[:,3:])


C:\Users\boroj\AppData\Local\Temp\ipykernel_8328\3331908067.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  cylinder_pq[:,3:] = multiply_quaternion(np.array(cylinder.get_quat()), cylinder_pq[:,3:])


In [9]:
# bottle randomization
bottle_pq = adapter.get_pose(bottle)
bottle_pq[:, 0] = np.random.uniform(-0.2, -0.5, size=(n_envs,))
bottle_pq[:, 1] = np.random.uniform(0.3, 0.55, size=(n_envs,))
bot_theta = np.random.uniform(-np.pi/6, np.pi/6, size=(n_envs,))
bottle_pq[:,3] = np.cos(bot_theta)
bottle_pq[:,4] =  np.sin(bot_theta)
bottle_pq[:,5] = bottle_pq[:,6] = np.zeros(n_envs)
bottle_pq[:,3:] = multiply_quaternion(np.array(bottle.get_quat()), bottle_pq[:,3:])

C:\Users\boroj\AppData\Local\Temp\ipykernel_8328\3039492635.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  bottle_pq[:,3:] = multiply_quaternion(np.array(bottle.get_quat()), bottle_pq[:,3:])


In [10]:
adapter.set_pose(obj=cube, pose=cube_pq)
adapter.set_pose(obj=cylinder, pose=cylinder_pq)
adapter.set_pose(obj=bottle, pose=bottle_pq)
scene.step()

[[ 0.23236743  0.35719955  0.24        0.94661665  0.          0.
  -0.32236147]
 [ 0.20942876  0.31924498  0.24        0.9166871   0.          0.
   0.39960584]
 [ 0.3909231   0.37243786  0.24        0.9991514   0.          0.
   0.04118745]
 [ 0.29430678  0.34030533  0.24        0.94862     0.          0.
   0.31641752]
 [ 0.35257122  0.5324244   0.24        0.9152026   0.          0.
   0.40299395]
 [ 0.47226995  0.5020301   0.24        0.98189336  0.          0.
  -0.18943463]
 [ 0.27478766  0.45835093  0.24        0.9177762   0.          0.
  -0.39709798]
 [ 0.32311487  0.5178651   0.24        0.9596882   0.          0.
  -0.2810669 ]
 [ 0.42666534  0.50091803  0.24        0.9970881   0.          0.
  -0.07625844]]
[[-0.4045037  -0.38080072  0.22       -0.15797679 -0.68923384 -0.15797679
   0.68923384]
 [-0.41518265 -0.42969766  0.22       -0.32964087 -0.6255693  -0.32964087
   0.6255693 ]
 [-0.20173803 -0.47575474  0.22        0.08095236 -0.7024576   0.08095236
   0.7024576 ]
 [-

In [28]:
import trimesh

In [34]:
cylinder.get_mass()

np.float64(0.0842790191109674)

In [37]:
cyl_verts = cylinder.get_verts()[0]
mesh = trimesh.Trimesh(vertices=cyl_verts)
obb = mesh.bounding_box_oriented
obb.vertices, obb.transform, obb.extents

(TrackedArray([[-0.45442322, -0.44983773,  0.23184559],
               [-0.31939692, -0.38450846,  0.23186167],
               [-0.47736111, -0.40243531,  0.25999301],
               [-0.34233481, -0.33710604,  0.26000909],
               [-0.46667713, -0.42449767,  0.17918502],
               [-0.33165083, -0.35916841,  0.17920111],
               [-0.48961502, -0.37709525,  0.20733244],
               [-0.35458872, -0.31176598,  0.20734853]]),
 TrackedArray([[-2.05220083e-01, -3.84147934e-01,  9.00175029e-01,
                -4.04505968e-01],
               [ 4.24377884e-01,  7.93863027e-01,  4.35528306e-01,
                -3.80801857e-01],
               [-8.81922973e-01,  4.71393529e-01,  1.07251166e-04,
                 2.19597057e-01],
               [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 1.00000000e+00]]),
 array([0.1702181 , 0.13807175, 0.08082407]))